In [1]:
import load_api_key
from langchain import PromptTemplate, LLMChain, OpenAI

## 通过LLMChain 拿到代码并执行

In [2]:
llm = OpenAI(model_name="text-davinci-003", max_tokens=2048, temperature=0.5)
multiply_by_python_prompt = PromptTemplate(template="请写一段Python代码，计算{question}?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_by_python_prompt, output_key="answer")
answer_code = math_chain.run({"question": "352乘以493"})

from langchain.utilities import PythonREPL

python_repl = PythonREPL()
result = python_repl.run(answer_code)
print(result)

173536



## 通过LLMMathChain算算术
gpt不擅长算术，但是可以通过LLMMathChain算算术

In [ ]:
from langchain import LLMMathChain

llm_math = LLMMathChain(llm=llm, verbose=True)
result = llm_math.run("请计算一下352乘以493是多少?")
print(result)

## 通过Google搜索查天气

In [9]:
from langchain.chains import LLMRequestsChain

template = """在 >>> 和 <<< 直接是来自Google的原始搜索结果.
请把对于问题 '{query}' 的答案从里面提取出来，如果里面没有相关信息的话就说 "找不到"
如果有温度的话，请使用摄氏单位转换。
请使用如下格式：
Extracted:<answer or "找不到">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)
requests_chain = LLMRequestsChain(llm_chain=LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT))
question = "今天上海的天气怎么样？"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+")
}
result = requests_chain(inputs)
print(result)
print(result['output'])

{'query': '今天上海的天气怎么样？', 'url': 'https://www.google.com/search?q=今天上海的天气怎么样？', 'output': ' 今天上海的天气为阴天，最高气温15℃，最低气温17℃，风力4-5级，最大降水机会为60%。'}
 今天上海的天气为阴天，最高气温15℃，最低气温17℃，风力4-5级，最大降水机会为60%。


## 通过TransformChain运行自定义函数

In [ ]:
import re


def parse_weather_info(weather_info) -> dict:
    # 将天气信息拆分成不同部分
    parts = weather_info.split('，')

    # 解析天气
    weather = parts[0].strip()

    # 解析温度范围，并提取最小和最大温度
    temperature_range = parts[1].strip().replace('°C', '').split('～')
    temperature_min = int(temperature_range[0])
    temperature_max = int(temperature_range[1])

    # 解析风向和风力
    wind_parts = parts[2].split(' ')
    wind_direction = wind_parts[0].strip()
    wind_force = wind_parts[1].strip()
    weather_dict = {
        'weather': weather,
        'temperature': {
            'min': temperature_min,
            'max': temperature_max,
        },
        'wind': {
            'direction': wind_direction,
            'level': wind_force
        }
    }
    return weather_dict


weather_dict = parse_weather_info(result['output'])
print(weather_dict)

## 通过VectorDBQA做问答

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import SpacyTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.document_loaders import TextLoader

llm = OpenAI(temperature=0)
loader = TextLoader('./data/ecommerce_faq.txt')
documents = loader.load()
# 通过 SpacyTextSplitter 将文档分段，确保每个分出来的 Document 都是一个完整的句子。
text_splitter = SpacyTextSplitter(chunk_size=256, pipeline="zh_core_web_sm")
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_documents(texts, embeddings)

# 通过 VectorDBQA 的 from_chain_type 定义了一个 LLM。
faq_chain = VectorDBQA.from_chain_type(llm=llm, vectorstore=docsearch, verbose=True)

FileNotFoundError: [Errno 2] No such file or directory: './data/ecommerce_faq.txt'